**Internsip|Summer 2024**

**IMPORTANT**

You can use this **Jupyter** notebook *symbols_update.ipynb* to complete the assignment. 

If you are not familiar with **Jupyter**, please feel free to develop your solution using standard **.py** file instead.

To complete the assignment, you will need to **pip install pandas** library.

Please upload all your work to public **GitHub** repository and share the link with us.

There are three .csv files that we want to upload sequentially to the database: **symbols_update_1.csv**, **symbols_update_2.csv**, and **symbols_update_3.csv**. For the sake of simplicity, the database is represented by another .csv file **database.csv**.

The goal it to complete **SymbolsUpdate** class with three functions:
- ***load_new_data_from_file*** : loads and processes data from **symbols_update_n.csv** file
- ***save_new_data*** : saves loaded data to the **database.csv** file
- ***get_data_from_database*** : returns the most recently updated data for every symbol in **database.csv**

In [1]:
# import key libraries 
import pandas as pd
import numpy as np
import datetime

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### Note on ISIN Handling

- ISIN is an International Securities Identification Number, a 12-digit alphanumeric, whhich is a unqiue identifier of a given security.
- The Excel file was compiled based on the data from the ISIN [website](https://www.isin.net/country-codes/) (hyperlinked).
- This will be used to identify the "country" entry in the database.csv.

In [2]:
# Load the country codes from the Excel file
country_codes_df = pd.read_excel('CounntryCodes.xlsx')

# Create the dictionary from the DataFrame for quick lookup
country_codes_dict = pd.Series(country_codes_df.Definition.values, index=country_codes_df['Code Value']).to_dict()

In [11]:
def extract_letters(isin):
    # extract the first two ISIN letters - the country code
    country_code = ''
    for char in isin:
        if char.isalpha():
            country_code += char
            if len(country_code) == 2:
                break

    # return the country name or 'Unknown' just in case
    return country_codes_dict.get(country_code, 'Unknown')

In [4]:
class SymbolsUpdate:
    def __init__(self):
        """
        Initialisation of the SymbolsUpdate class with the database path.
        """
        self.database_file = 'database.csv'

    def load_new_data_from_file(self, file_path: str) -> pd.DataFrame:
        """
        Loads, processes data from specified path, returns as a DataFrame.

        Parameters:
        file_path (str): The path to the symbols update CSV file.

        Returns:
        pd.DataFrame: A DataFrame containing the processed data.
        """
        # Initialise a DataFrame
        new = pd.DataFrame()

        # Read the CSV
        file = pd.read_csv(file_path)

        # Iterate through each row
        for i in range(len(file)):
            symbol = file['symbol'][i]
            country_id = extract_letters(file['isin'][i])
            hold = file['hold'][i]
            cusip = file['cusip'][i]
            isin = file['isin'][i]
            current_time = datetime.datetime.now()
            formatted_time = current_time.strftime('%Y-%m-%d %H:%M:%S.%f')

            # Create rows for 'cusip' and 'isin'
            row1 = {'symbol': symbol, 'hold': hold, 'country': country_id, 'item': 'cusip', 'item_value': cusip, 'updatedby': agent, 'updatetime': formatted_time}
            row2 = {'symbol': symbol, 'hold': hold, 'country': country_id, 'item': 'isin', 'item_value': isin, 'updatedby': agent, 'updatetime': formatted_time}

            # Combine the rows
            pair = pd.DataFrame([row1, row2])

            # Append to DataFrame
            new = pd.concat([new, pair], ignore_index=True)

        return new

    def save_new_data(self, input_data: pd.DataFrame):
        """
        Adds the provided DataFrame to the database file.

        Parameters:
        input_data (pd.DataFrame): The DataFrame to be added.
        """
        # Load database
        DB = pd.read_csv(self.database_file)

        # Check if empty
        if DB.empty:
            # Save data directly if the database is empty
            input_data.to_csv(self.database_file, index=False)
        else:
            # Concatenate the database with the data save
            pd.concat([DB, input_data], ignore_index=True).to_csv(self.database_file, index=False)

    def get_data_from_database(self) -> pd.DataFrame:
        """
        Returns the up to date data for each symbol from the database file.

        Returns:
        pd.DataFrame: A DataFrame containing the most recent updates for each symbol.
        """
        # Load database
        DB = pd.read_csv(self.database_file)

        # Sort DataFrame by 'symbol' and 'updatetime'
        DB = DB.sort_values(by=['symbol', 'updatetime'], ascending=[True, False])

        # Group by 'symbol' and select the most recent rows per symbol
        DB = DB.groupby('symbol').head(2)

        # Sort in alphabetical order
        DB = DB.sort_values(by='symbol')

        return DB

Running the model should return the following table:

In [12]:
initialise_database()

# content for the 'updatedby' column
agent = 'petroineos'

The file 'database.csv' was empty and has been initialised.


In [13]:
su = SymbolsUpdate()
new_data = su.load_new_data_from_file('symbols_update_1.csv')
su.save_new_data(new_data)
new_data = su.load_new_data_from_file('symbols_update_2.csv')
su.save_new_data(new_data)
new_data = su.load_new_data_from_file('symbols_update_3.csv')
su.save_new_data(new_data)

EmptyDataError: No columns to parse from file

In [ ]:
su.get_data_from_database()

In [ ]:
from IPython.display import Image
Image('output.png')

Please note:  
- *isin* and *cusip* columns from **symbols_update_n.csv** files are stored as item/item_value pairs in **database.csv**
- *country id* is derived from the *isin* field (e.g. US01222911 is US, GB12222201 is GB, etc…)
- The most recent update for each symbol is returned based on the **updatetime** column

**Good luck!!!**